# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать. Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке.

Цель исследования: проанализировать базу данных с книгами, издательствами, авторами, а так же пользовательскими обзорами книг, чтобы сформулировать ценностное предложение для нового продукта.

Исследование состоит из следующих этапов:
1. Обзор данных;
2. Посчитать, сколько книг вышло после 1 января 2000 года;
3. Для каждой книги посчитать количество обзоров и среднюю оценку;
4. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц для исключения из анализа брошюр;
5. Определить автора с самой высокой средней оценкой книг;
6. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок;
7. Выводы по исследованию.

### Обзор данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

query = '''
SELECT *
FROM books
LIMIT 5
        '''

# чтобы выполнить SQL-запрос, используем Pandas
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [2]:
query = '''
SELECT *
FROM authors
LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [3]:
query = '''
SELECT *
FROM publishers
LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [4]:
query = '''
SELECT *
FROM ratings
LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
query = '''
SELECT *
FROM reviews
LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Посчитаем, сколько книг вышло после 1 января 2000 года:

In [6]:
query = '''
SELECT COUNT(*) 
FROM books
WHERE publication_date > '2000-01-01' 
        '''

pd.io.sql.read_sql(query, con = engine)

,count
0,819


После 1 января 2000 года вышло 819 книг.

### Для каждой книги посчитаем количество обзоров и среднюю оценку:

In [7]:
query = '''
SELECT b.title,
       COUNT(DISTINCT rew.review_id) AS count_reviews, 
       ROUND(AVG(rat.rating), 2) AS avg_rating
FROM books AS b
LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
LEFT JOIN reviews AS rew ON b.book_id = rew.book_id
GROUP BY b.book_id
ORDER BY avg_rating DESC,
         count_reviews DESC
        '''

pd.io.sql.read_sql(query, con = engine)

,title,count_reviews,avg_rating
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,The Big Bad Wolf (Alex Cross #9),2,5.00
4,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


Всего в данных у нас 1000 книг. Минимальный рейтинг книги - 1.50, максимальный - 5.00. А количество обзоров у книг в промежутке от 0 до 7. 
Можем выделить ТОП-5 книг по средней оценке и количеству обзоров: 
1. A Dirty Job (Grim Reaper #1) (средний рейтинг - 5.00, 4 обзора) 
2. School's Out—Forever (Maximum Ride #2) (средний рейтинг - 5.00, 3 обзора) 
3. Moneyball: The Art of Winning an Unfair Game (средний рейтинг - 5.00, 3 обзора) 
4. The Big Bad Wolf (Alex Cross #9) (средний рейтинг - 5.00, 2 обзора) 
5. Welcome to Temptation (Dempseys #1) (средний рейтинг - 5.00, 2 обзора)

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры.

In [8]:
query = '''
SELECT p.publisher,
       COUNT(b.book_id)
FROM publishers AS p
JOIN books AS b ON p.publisher_id=b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id
ORDER BY COUNT(b.book_id) DESC
LIMIT 1
        '''

pd.io.sql.read_sql(query, con = engine)

,publisher,count
0,Penguin Books,42


Таким образом, издательство, выпустившее наибольшее число книг толще 50 страниц - Penguin Books. Оно выпустило 42 книги.

### Определим автора с самой высокой средней оценкой книг — будем учитывать только книги с 50 и более оценками.

In [11]:
query = '''
SELECT avg.author,
       ROUND(AVG(avg.avg_rating_1), 2) AS avg_rating
FROM  (SELECT 
            a.author,
            AVG(r.rating) AS avg_rating_1
        FROM books AS b
            LEFT JOIN authors AS a ON b.author_id = a.author_id
            LEFT JOIN ratings AS r ON b.book_id = r.book_id
        GROUP BY b.book_id, a.author_id
        HAVING COUNT(r.rating_id) >=50) AS avg
GROUP BY avg.author
ORDER BY avg_rating DESC
LIMIT 1
        '''

pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.28


Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré, средняя оценка его книг - 4.28.

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок:

In [12]:
query = '''
SELECT ROUND(AVG(r.count_reviews)) AS avg_review
FROM (SELECT username,
             COUNT(review_id) AS count_reviews
      FROM reviews
      WHERE username IN
          (SELECT username
           FROM ratings
           GROUP BY username
           HAVING COUNT(rating_id) > 50)
           GROUP BY username) AS r
        '''

pd.io.sql.read_sql(query, con = engine)

,avg_review
0,24.0


В среднем, у пользователей, которые поставили больше 50 оценок - 24 обзора.

### Выводы по исследованию

- После 1 января 2000 года вышло 819 книг;
- Всего в данных у нас 1000 книг. Минимальный рейтинг книги - 1.50, максимальный - 5.00. А количество обзоров у книг в промежутке от 0 до 7;
- Можем выделить ТОП-5 книг по средней оценке и количеству обзоров: 
1. A Dirty Job (Grim Reaper #1) (средний рейтинг - 5.00, 4 обзора) 
2. School's Out—Forever (Maximum Ride #2) (средний рейтинг - 5.00, 3 обзора) 
3. Moneyball: The Art of Winning an Unfair Game (средний рейтинг - 5.00, 3 обзора) 
4. The Big Bad Wolf (Alex Cross #9) (средний рейтинг - 5.00, 2 обзора) 
5. Welcome to Temptation (Dempseys #1) (средний рейтинг - 5.00, 2 обзора)
- Издательство, выпустившее наибольшее число книг толще 50 страниц - Penguin Books. Оно выпустило 42 книги;
- Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré, средняя оценка его книг - 4.28;
- В среднем, у пользователей, которые поставили больше 50 оценок - 24 обзора.